In [1]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-2oCiQGD-PYRYC7KzN0yiXPGvUwTRlZota34yB4Axyt-bUX_RCRrVT7w3XqpmMRf0MMmTvGl_w_Ztb41FXYNPgA"

In [2]:
!pip install -q langchain-openai langchain-community langchain-core requests duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
import requests

In [4]:
from langchain_community.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

In [5]:
#custom tools
@tool
def get_weather_data(city: str) -> str:
  """
  This function fetches the current weather data for a given city
  """
  url = f'https://api.weatherstack.com/current?access_key=4d1d8ae207a8c845a52df8a67bf3623e&query={city}'

  response = requests.get(url)

  return response.json()

In [6]:
llm = ChatOpenAI()

In [7]:
#Step1: import necessary library
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub     #hub is mainly used to load prebuilt prompts

In [8]:
# Step 2: Pull the ReAct prompt from LangChain Hub
prompt = hub.pull("hwchase17/react")


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [9]:
# Step 3: Create the ReAct agent manually with the pulled prompt
agent = create_react_agent(
    llm=llm,
    tools=[search_tool, get_weather_data],
    prompt=prompt
)

In [11]:
# Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=[search_tool, get_weather_data],
    verbose=True
)     #it takes the agent + tools and actually execute the task

In [14]:
# Step 5: Invoke
response = agent_executor.invoke({"input": "Find the capital of UTTARAKhAND, then find it's current weather condition and the air quality"})
print(response)    #agent scratchpad (it gives the entire thought trace)



> Entering new AgentExecutor chain...
First, I need to find the capital of Uttarakhand. Once I have that information, I can get the current weather data and air quality for that city.
Action: duckduckgo_search
Action Input: "Capital of Uttarakhand"Uttarakhand has a total area of 53,483 km 2 (20,650 sq mi), equal to 1.6% of the total area of India. Dehradun serves as the state capital, with Nainital being the judicial capital. The state is divided into two divisions, Garhwal and Kumaon, with a total of 13 districts. The forest cover in the state is 45.4% of the state's geographical area. Its capital is the northwestern city of Dehra Dun. On November 9, 2000, the state of Uttaranchal—the 27th state of India—was carved out of Uttar Pradesh, and in January 2007 the new state changed its name to Uttarakhand, meaning "northern region," which was the traditional name for the area. Area 19,739 square miles (51,125 square km). Pop. This city, the capital of Uttarakhand, attracts visitors with

In [15]:
response['output']

'The capital of Uttarakhand is Dehradun. The current weather condition in Dehradun is sunny with a temperature of 37°C and an air quality index of 4.'